In [14]:
from bs4 import BeautifulSoup as bs 
import requests
import pandas as pd
import numpy as np
from time import sleep
import pprint

In [15]:
#WITHOUT HEADERS WE WILL GET ERROR 403 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#LISTS VALUES FROM SCRAPE
movie_name = []
year = []
duration = [] 
rate =[]
picture_url = []
category = []
director = []

In [16]:
#movie_name,year,duration,category,rate,director,actors,picture_url
pages=[1,2,3,4,5,6,7,8,9,10]
for num_page in pages:
    imdb_movies_url = 'https://www.imdb.com/list/ls048276758/?sort=moviemeter,asc&st_dt=&mode=detail&page='+str(num_page)
    page = requests.get(imdb_movies_url,headers=headers)
    soup = bs(page.content,'html.parser')
    data_per_movie = soup.findAll('div', attrs = {'class': 'lister-item mode-detail'})
    #Must put sleep - without the request may crash
    sleep(4)
   
    for store in data_per_movie:
        if store.h3.a.text is not None:
            name = store.h3.a.text
            movie_name.append(name)
        else: movie_name.append(None)


        if store.h3.find('span', class_ = "lister-item-year text-muted unbold") is not None:
            year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(','').replace(')','')
            year.append(year_of_release)
        else:year.append(None)

        if store.p.find('span', class_ = 'runtime') is not None:            
            time = int(store.p.find('span', class_ = 'runtime').text.replace(' min',''))
            duration.append(time)
        else: duration.append(None)

        if store.p.find('span', class_ = 'genre') is not None:     
            categories = store.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',')
            category.append(categories)
        else:category.append(None)

        if store.find('span', class_ = "ipl-rating-star__rating") is not None:
            rating = float(store.find('span', class_ = "ipl-rating-star__rating").text)
            rate.append(rating)
        else:rate.append(None)

        #Success only the first director
        if store.findAll('p',class_ = 'text-muted text-small')[1] is not None:
            directors_sections = store.findAll('p',class_ = 'text-muted text-small')[1]
            if directors_sections.find('a') is not None:
                director_name = directors_sections.find('a').text
                director.append(director_name)
            else:director.append(None)
        else:director.append(None)

        if store.find('div', class_ = "lister-item-image ribbonize") is not None:
            pic_section = store.find('div', class_ = "lister-item-image ribbonize")
            if pic_section.find('img', class_='loadlate')['loadlate'] is not None:           
                pic_url = pic_section.find('img', class_='loadlate')['loadlate']
                picture_url.append(pic_url)
            else:picture_url.append(None)
        else:picture_url.append(None)

        
      #TRY-EXCEPT not suit here because we need to fill the values per section   
  
  


In [17]:
print(len(movie_name))
print(len(year))
print(len(duration))
print(len(category))
print(len(director))
print(len(rate))
print(len(picture_url))

1000
1000
1000
1000
1000
1000
1000


In [18]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name,\
                            "Year of Release" : year,\
                            "Duration (Min)": duration,\
                            "Genres": category,\
                            "Director" : director,\
                            "Movie Rating": rate,\
                            "Photo URL":picture_url
                            })

In [22]:
movie_list['Photo URL']

0      https://m.media-amazon.com/images/M/MV5BNDE3OD...
1      https://m.media-amazon.com/images/M/MV5BM2MyNj...
2      https://m.media-amazon.com/images/M/MV5BMTMxNT...
3      https://m.media-amazon.com/images/M/MV5BNGNhMD...
4      https://m.media-amazon.com/images/M/MV5BMjAxMz...
                             ...                        
995    https://m.media-amazon.com/images/M/MV5BOTMwMD...
996    https://m.media-amazon.com/images/M/MV5BMTE1MT...
997    https://m.media-amazon.com/images/M/MV5BOTgwMD...
998    https://m.media-amazon.com/images/M/MV5BZGRkMW...
999    https://m.media-amazon.com/images/M/MV5BOWNjYz...
Name: Photo URL, Length: 1000, dtype: object

In [28]:
#save to excl just for verify by hand
movie_list.to_excel('movies.xlsx')

In [64]:
# save the data in csv format
movie_list.to_csv("movies.csv")